In [2]:
# import packages 
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
# import PCA-related scikitlearn modules ??
# may just need the modules necessary to turn the PCAs into a multi-linear model

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
# import data

# kitchen sink models

gunData = pd.read_excel('firearm_data_cleaned_new.xlsx')
gunData.head()

data1 = gunData[['year', 'state', 'rate', 'deaths', 'law_strength_score',
                    'restrictive_laws', 'permissive_laws', 'total_law_changes',                             
                    'law_strength_change', 'unique_law_classes']].dropna()
    # state is enough, omitting state_name
    # omitting rate_change
data1 = data1.query("state!='District of Columbia'")
    # there is only one datapoint for District of Columbia, flummoxes train_test_split. dropping.

bigX_1 = data1[['year', 'state', 'deaths', 'law_strength_score',
                    'restrictive_laws', 'permissive_laws', 'total_law_changes',                             
                    'law_strength_change', 'unique_law_classes']]
    # state is enough, omitting state_name
    # omitting rate_change

bigY1 = data1['rate']

categories1 = ['year', 'state']
numers1 = ['deaths', 'law_strength_score', 'restrictive_laws', 'permissive_laws', 'total_law_changes',                           
                    'law_strength_change', 'unique_law_classes']

# dummify 'state'
#X = pd.get_dummies(data2["smoker"], drop_first = True, dtype = float)
bigX_11 = pd.get_dummies(bigX_1['state'], drop_first=True, dtype=float)
bigX_12 = pd.concat([bigX_1, bigX_11], axis=1)
bigX_12['AK'] = bigX_12['state'] == 'AK'
bigX_1 = bigX_12.drop('state', axis=1)
# X.columns = X.columns.astype(str)

# dummify 'year'
temp = pd.get_dummies(bigX_1['year'], drop_first=True, dtype=float)
temp2 = pd.concat([bigX_1, temp], axis=1)
temp2[2015] = temp2['year']==2015
bigX_1 = temp2.drop('year', axis=1)

bigX_1.columns = bigX_1.columns.astype(str)


In [5]:
bigX_1.dtypes

deaths                  int64
law_strength_score      int64
restrictive_laws        int64
permissive_laws         int64
total_law_changes       int64
                       ...   
2020                  float64
2021                  float64
2022                  float64
2023                  float64
2015                     bool
Length: 66, dtype: object

In [6]:
data2 = gunData[['rate', 'year', 'state', 'strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']].dropna()
data2 = data2.query("state!='District of Columbia'")

bigX_2 = data2[['state', 'year', 'strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']]

categories2 = ['state', 'year']
numers2 = ['strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']

bigY2 = data2['rate']

# dummify 'state'
temp = pd.get_dummies(bigX_2['state'], drop_first=True, dtype=float)
temp2 = pd.concat([bigX_2, temp], axis=1)
temp2['AK'] = temp2['state'] == 'AK'
bigX_2 = temp2.drop('state', axis=1)

# dummify 'year'
temp = pd.get_dummies(bigX_2['year'], drop_first=True, dtype=float)
temp2 = pd.concat([bigX_2, temp], axis=1)
temp2[2015] = temp2['year']==2015
bigX_2 = temp2.drop('year', axis=1)

bigX_2.columns = bigX_2.columns.astype(str)

In [7]:
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(bigX_1, bigY1, test_size=0.2, random_state=123)
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(bigX_2, bigY2, test_size=0.2, random_state=123)

In [8]:
type(x_train_1)

pandas.core.frame.DataFrame

In [ ]:
"""
Apply PCA (IAW notebook 17 and guidance from genAI)
"""
# center TRAINING data and calculate training data mean
x_train_1 = x_train_1.astype(float)
x_train_2 = x_train_2.astype(float)

mu_train1 = x_train_1.mean()
mu_train2 = x_train_2.mean()

Xc_1t = x_train_1 - mu_train1
Xc_2t = x_train_2 - mu_train2

# generate variance-covariance matrix
cov_1 = np.cov(Xc_1t.T)
cov_2 = np.cov(Xc_2t.T)

# eigendecomposition >> eigenvalues/eigenvectors
eigvals_1, eigvecs_1 = np.linalg.eig(cov_1)
eigvals_2, eigvecs_2 = np.linalg.eig(cov_2)

# sort the eigenvalues and eigenvectors
order1 = np.argsort(eigvals_1)[::-1]
order2 = np.argsort(eigvals_2)[::-1]

eigval_1_sorted = eigvals_1[order1]
eigval_2_sorted = eigvals_2[order2]

eigvec_1_sorted = eigvecs_1[order1]
eigvec_2_sorted = eigvecs_2[order2]

# isolate desired eigenvectors >> each column is one eigenvector
candidates1 = eigvec_1_sorted[:,:4]
candidates2 = eigvec_2_sorted[:,:4]

# project (centered) training data
pc_train_1 = Xc_1t @ candidates1
pc_train_2 = Xc_2t @ candidates2

# center test data with training data mean
Xc_1test = x_test_1 - mu_train1
Xc_2test = x_test_2 - mu_train2

# project (ceentered) test data
pc_test_1 = Xc_1test @ candidates1
pc_test_2 = Xc_2test @ candidates2

# regress (train model) >> LinearRegression() >> model.fit()
pc_model1 = LinearRegression()
pc_model1.fit(pc_train_1, y_train_1)

pc_model2 = LinearRegression()
pc_model2.fit(pc_train_2, y_train_2)

# predict (test model) using centered test data >> model.predict()
predictions1 = pc_model1.predict(pc_test_1)
predictions2 = pc_model2.predict(pc_test_2)

In [10]:
# evaluate performance
pc_data1 = pd.DataFrame({"actual": y_test_1, "predicted": predictions1})
pc_data2 = pd.DataFrame({"actual": y_test_2, "predicted": predictions2})

px.scatter(pc_data1, x='predicted', y='actual', trendline='ols')

In [11]:
px.scatter(pc_data2, x='predicted', y='actual', trendline='ols')

In [12]:
type(Xc_1test)

pandas.core.frame.DataFrame

In [15]:
# inputs to .score are both x_test and y_test, in this case the centered X test data
r2_1 = pc_model1.score(pc_test_1, y_test_1)
n = x_test_1.shape[0]
p = x_test_1.shape[1]
adj_r2_1 = 1 - (1 - r2_1) * (n - 1) / (n - p -1)
print(f"Adjusted r^2 for PC model 1: {adj_r2_1}")

Adjusted r^2 for PC model 1: -1.4809854668171996


In [16]:
r2_2 = pc_model2.score(pc_test_2, y_test_2)
n = x_test_2.shape[0]
p = x_test_2.shape[1]
adj_r2_2 = 1 - (1 - r2_2) * (n - 1) / (n - p -1)
print(f"Adjusted r^2 for PC model 2: {adj_r2_2}")


Adjusted r^2 for PC model 2: -1.6890152630239257


In [ ]:
mse_calc1_1 = mean_squared_error(y_test_1, predictions1)
root_mse1_1 = mse_calc1_1 ** 0.5
root_mse1_1

4.651766996463598

In [ ]:
mse_calc1_1 = mean_squared_error(y_test_2, predictions2)
root_mse1_1 = mse_calc1_1 ** 0.5
root_mse1_1

4.280241577256119